In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.random.seed(1234)
df_1 = pd.DataFrame({'A': ['a', 'b', 'c', 'd']*5,
                     'B': np.random.randn(20),
                     'C': np.random.randint(7, size=20)})
df_2 = pd.read_csv('../data/pandas_2_df_2.csv')
df_3 = pd.read_csv('../data/pandas_2_df_3.csv')
df_4 = pd.read_csv('../data/pandas_2_df_4.csv')

# Assignment 2.1
Find the minimum, maximum and median values of all columns for each unique value in column `A` in DataFrame `df_1`.The `groupby` method on a pandas DataFrame has the possibility to group by the index values by using `.groupby(level=0)`
<details><summary>Hint 1</summary>
    <p>
        Use the `groupby` method on a DataFrame to compute aggregate values for each unique value in a column
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        It is possible to apply multiple aggregation functions to a `groupby` object by using the `agg` method and providing the function names as a list of strings
    </p>
</details>

In [ ]:
df_1.groupby('A').agg(['min', 'max', 'median'])

# Assignment 2.2
For each unique combination of values from columns `A` and `C`, find the average of the values in column `B`. Return a DataFrame with columns `A`, `B_avg` and `C`, where `B_avg` holds the result of the average over the `B` column, while `A` and `C` hold the values for the unique combinations.
<details><summary>Hint 1</summary>
    <p>
        The `groupby` method on a pandas DataFrame can take a list of column names as input, and will group by every unique combination of values in the listed columns
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Applying a single aggregate function to a `groupby` object returns a Series. This series has a method `rename` for renaming it
    </p>
</details>

In [ ]:
df_1.groupby(['A','C'])['B'].mean().rename('B_avg').reset_index()

# Assignment 2.3
The DataFrame `df_2` contains students' ID (`student_id`), their ID's of the courses (`course_id`) and their grade (`grade`). Compute, for every student, the ratio of grades that are A's or B's.
<details><summary>Hint 1</summary>
    <p>
        The `groupby` method on a pandas DataFrame has the possibility to group by the index values by using `.groupby(level=0)`
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Applying a numerical aggregate function to a Boolean Series, leads pandas interpreting True as a 1 and False as a 0
    </p>
</details>

In [ ]:
df_2.groupby('student_id')['grade'].apply(lambda s: s.isin(['A', 'B']).mean())

# Assignment 2.4
The DataFrame `df_3` contains a series of columns named after the pattern `variable name`_`time lag`, e.g. the variable `A` has current time unit's value, plus that of the last three time units, stored in `A_0`, `A_1`, `A_2`, `A_3`. Compute the average over all time lags for every variable name for every record. Return a DataFrame with the `customer_id` as index and the average of every variable as columns.

All lag variables in DataFrame ends with _ and a single number.
<details><summary>Hint 1</summary>
    <p>
        It is possible to group by columns by specifying `axis=1` as part of the input to the method
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        One can control which rows or columns are grouped together by supplying a list of values to the `by` parameter in the method
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        The python function `map` can apply a function to every element of a list
    </p>
</details>

In [ ]:
column_groups = list(map(lambda s: s[:-2],df_3.set_index('customer_id').columns.tolist()))
df_3.set_index('customer_id').groupby(by=column_groups, axis=1).mean()

# Assignment 2.5
All the grades in `df_2` has a grading date in column `grade_date`. Find out which address the grades were sent to after each grading for every student. Use the data in `df_4` to get the address on file for the grading date.

Return a DataFrame where each row holds a student ID, the grading date and a list of course IDs graded at that date, along with the address the grades were sent to.

In [ ]:
valid_address = (df_4
                 .groupby('student_id')
                 .apply(lambda df:
                        pd.concat([df[['student_id', 'address', 'valid_from']],
                                   df['valid_from']
                                   .shift(-1)
                                   .rename('valid_to')
                                   .to_frame()],
                                  axis=1)))

courses_per_grading = (df_2.groupby(['student_id',
                                     'grade_date'])['course_id']
                       .apply(lambda s: s.tolist())
                       .reset_index())

tmp = pd.merge(left=courses_per_grading,
               right=valid_address,
               on='student_id')

tmp.loc[(tmp['grade_date']>=tmp['valid_from'])&
        ((tmp['grade_date']<tmp['valid_to'])|
         (tmp['valid_to'].isnull())),
        ['student_id', 'grade_date', 'course_id', 'address']]